In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge, BayesianRidge
from sklearn.datasets import make_regression

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
team_data = pd.read_csv('../Stats_competition-/final_opponent_and_team_data.csv')
team_data['Location'] = np.where(team_data['Location'] == 'N', 0, np.where(team_data['Location'] == 'H', 1, -1))
columns_to_convert = ['Location','ADJO', 'ADJD', 'EFG%', 'TO%', 'OR%', 'FTR', 'Opp EFG%', 'Opp TO%', 'Opp OR%', 'Opp FTR']
for col in columns_to_convert:
    team_data[col] = pd.to_numeric(team_data[col], errors='coerce')
    
cleanDate = team_data.dropna()

In [3]:
# Removing columns not needed for predictions
feature_cols = cleanDate.columns.difference(['Date', 'Team', 'Opponent', 'Team_score', 'Opponent_score',
                                             '2P', '3P', 'Opp 2P', 'Opp 3P',
                                             'EFF', 'Opp EFF', 'WAB'])

# Defining the feature matrix (X) and target matrix (y)
X = cleanDate[feature_cols]
y = cleanDate[['Team_score', 'Opponent_score']]

display(X.head())
display(y.head())

,ADJD,ADJO,EFG%,FTR,Location,OR%,Opp EFG%,Opp FTR,Opp OR%,Opp TO%,TO%,opp_adj_d,opp_adj_o
0,95.2,125.3,64.3,31.7,1,35.5,39.5,31.6,19.4,19.0,17.7,110.8,108.9
1,92.3,124.7,61.3,25.4,1,43.6,39.1,14.1,23.3,18.3,11.3,119.0,107.2
2,86.4,106.8,42.3,23.9,0,25.0,47.6,38.1,23.3,14.6,9.3,88.1,119.6
3,56.9,124.7,61.3,17.7,1,45.2,28.9,5.3,29.3,29.5,15.5,115.5,69.8
4,75.9,111.7,50.0,21.3,-1,35.1,45.3,20.8,16.7,22.2,20.7,90.4,98.9


,Team_score,Opponent_score
0,96,62
1,100,58
2,72,77
3,86,35
4,69,55


## Train Model

In [4]:
## Train Bayesian Ridge regression models for multiple target variables
def trainModel(X, y):
    models = {}
    for target in y.columns:
        print(f"Training model for {target}...")

        # Initialize the model
        model = BayesianRidge()
        
        # Train the model
        model.fit(X, y[target])
        models[target] = model

        print(f"Model for {target} trained successfully.\n")

    return models

model1 = trainModel(X, y)
print(model1)

Training model for Team_score...
Model for Team_score trained successfully.

Training model for Opponent_score...
Model for Opponent_score trained successfully.

{'Team_score': BayesianRidge(), 'Opponent_score': BayesianRidge()}


In [5]:
display(X.head())
display(y.head())

,ADJD,ADJO,EFG%,FTR,Location,OR%,Opp EFG%,Opp FTR,Opp OR%,Opp TO%,TO%,opp_adj_d,opp_adj_o
0,95.2,125.3,64.3,31.7,1,35.5,39.5,31.6,19.4,19.0,17.7,110.8,108.9
1,92.3,124.7,61.3,25.4,1,43.6,39.1,14.1,23.3,18.3,11.3,119.0,107.2
2,86.4,106.8,42.3,23.9,0,25.0,47.6,38.1,23.3,14.6,9.3,88.1,119.6
3,56.9,124.7,61.3,17.7,1,45.2,28.9,5.3,29.3,29.5,15.5,115.5,69.8
4,75.9,111.7,50.0,21.3,-1,35.1,45.3,20.8,16.7,22.2,20.7,90.4,98.9


,Team_score,Opponent_score
0,96,62
1,100,58
2,72,77
3,86,35
4,69,55


## Test model using test data

In [6]:
def predictModel(models, data):
    # Preprocess the data
    print("Preprocessing data for prediction...")
    data['Location'] = np.where(data['Location'] == 'Neutral', 0, 
                                np.where(data['Location'] == 'Home', 1, -1))
    
    columns_to_convert = ['ADJD', 'ADJO', 'EFG%', 'FTR', 'Location', 'OR%', 
                        'Opp EFG%', 'Opp FTR', 'Opp OR%', 'Opp TO%', 'TO%',
                        'opp_adj_d', 'opp_adj_o']
    
    for col in columns_to_convert:
        data[col] = pd.to_numeric(data[col], errors='coerce')
    
    
    # Extract the features for prediction
    X = data[columns_to_convert]

    # Make predictions using the models
    print("Making predictions...")
    predictions = pd.DataFrame()
    for target, model in models.items():
        print(f"Predicting {target}...")
        predictions[target] = model.predict(X)

    print("Predictions completed.\n")
    
    teams = data[['Team', 'Opponent']]
    finalPredictions = pd.concat([teams, predictions], axis=1)
    
    return finalPredictions

# Example usage:
predict_data = pd.read_csv('../Stats_competition-/basketball_games_data.csv')
predictions = predictModel(model1, predict_data)
display(predictions)


Preprocessing data for prediction...
Making predictions...
Predicting Team_score...
Predicting Opponent_score...
Predictions completed.



,Team,Opponent,Team_score,Opponent_score
0,Auburn,Purdue,89.430534,79.165254
1,Houston,Texas A&M Corpus Christi,81.128646,72.195457
2,Duke,Georgia Tech,79.623114,69.422654
3,Gonzaga,Bucknell,85.874906,66.704418
4,Florida,North Florida,83.399218,72.800038
5,Kentucky,Ohio State,84.551605,77.921747
6,Marquette,Xavier,80.815527,75.748384
7,UCLA,North Carolina,80.615577,76.135407
8,Maryland,Syracuse,85.109556,73.897145
9,Connecticut,Butler,86.992765,76.944019


# Backtesting Models

In [7]:
# Train and evaluate Bayesian Ridge regression models for multiple target variables.
def baseModel(X_train, y_train, X_test, y_test, threshold=6):
    models = {}
    for target in y_train.columns:
        print(f"Training model for {target}...")

        # Initialize the model
        model = BayesianRidge()

        # Train the model
        model.fit(X_train, y_train[target])
        models[target] = model

        # Make predictions
        y_pred = model.predict(X_test)

        # Evaluate accuracy within the threshold
        accuracy = (abs(y_pred - y_test[target]) <= threshold).mean() * 100
        print(f"Accuracy for {target} within {threshold} points: {accuracy:.2f}%")

        # Evaluate RMSE
        rmse = mean_squared_error(y_test[target], y_pred, squared=False)
        print(f"RMSE for {target}: {rmse:.4f}\n")

        # Optional: Uncomment to print predictions
        # print("Actual:", y_test[target].values)
        # print("Predicted values:", y_pred)

    return models


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_main = baseModel(X_train, y_train, X_test, y_test)
print(model_main)

Training model for Team_score...
Accuracy for Team_score within 6 points: 67.21%
RMSE for Team_score: 5.6670

Training model for Opponent_score...
Accuracy for Opponent_score within 6 points: 68.85%
RMSE for Opponent_score: 5.4352

{'Team_score': BayesianRidge(), 'Opponent_score': BayesianRidge()}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [8]:
def EnsembleModel(X_train, y_train, X_test, y_test, threshold=6):
    """
    Train and evaluate a StackingRegressor for multiple target variables.

    Parameters:
    X_train (DataFrame): Training feature set.
    y_train (DataFrame): Training target set (multi-target).
    X_test (DataFrame): Test feature set.
    y_test (DataFrame): Test target set (multi-target).
    threshold (int, optional): Threshold for accuracy evaluation. Default is 6.

    Returns:
    dict: A dictionary containing trained ensemble models for each target variable.
    """
    models = {}
    for target in y_train.columns:
        print(f"Training StackingRegressor model for {target}...")

        # Define base models and stacking regressor
        base_models = [
            ('ridge', Ridge()),
            ('bayesian_ridge', BayesianRidge())
        ]
        stacking_model = StackingRegressor(estimators=base_models, final_estimator=BayesianRidge())

        # Train the stacking model
        stacking_model.fit(X_train, y_train[target])
        models[target] = stacking_model

        # Make predictions
        y_pred = stacking_model.predict(X_test)

        # Evaluate accuracy within the threshold
        accuracy = (abs(y_pred - y_test[target]) <= threshold).mean() * 100
        print(f"Accuracy for {target} within {threshold} points: {accuracy:.2f}%")

        # Evaluate RMSE
        rmse = mean_squared_error(y_test[target], y_pred, squared=False)
        print(f"RMSE for {target}: {rmse:.4f}\n")

        # Optional: Uncomment to print predictions
        # print("Actual:", y_test[target].values)
        # print("Predicted values:", y_pred)

    return models

# Example Usage
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model2 = EnsembleModel(X_train, y_train, X_test, y_test)
print(model2)


Training StackingRegressor model for Team_score...
Accuracy for Team_score within 6 points: 67.21%
RMSE for Team_score: 5.6741

Training StackingRegressor model for Opponent_score...
Accuracy for Opponent_score within 6 points: 68.85%
RMSE for Opponent_score: 5.3860

{'Team_score': StackingRegressor(estimators=[('ridge', Ridge()),
                              ('bayesian_ridge', BayesianRidge())],
                  final_estimator=BayesianRidge()), 'Opponent_score': StackingRegressor(estimators=[('ridge', Ridge()),
                              ('bayesian_ridge', BayesianRidge())],
                  final_estimator=BayesianRidge())}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [9]:
#Train and evaluate a StackingRegressor combining Bayesian Ridge and Random Forest for multiple target variables.

def EnsembleModelWithRF(X_train, y_train, X_test, y_test, threshold=6):
    models = {}
    for target in y_train.columns:
        print(f"Training StackingRegressor model for {target}...")

        # Define base models and stacking regressor
        base_models = [
            ('bayesian_ridge', BayesianRidge()),
            ('random_forest', RandomForestRegressor(n_estimators=100, random_state=42))
        ]
        stacking_model = StackingRegressor(estimators=base_models, final_estimator=BayesianRidge())

        # Train the stacking model
        stacking_model.fit(X_train, y_train[target])
        models[target] = stacking_model

        # Make predictions
        y_pred = stacking_model.predict(X_test)

        # Evaluate accuracy within the threshold
        accuracy = (abs(y_pred - y_test[target]) <= threshold).mean() * 100
        print(f"Accuracy for {target} within {threshold} points: {accuracy:.2f}%")

        # Evaluate RMSE
        rmse = mean_squared_error(y_test[target], y_pred, squared=False)
        print(f"RMSE for {target}: {rmse:.4f}\n")

    return models

# Example Usage
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model3 = EnsembleModelWithRF(X_train, y_train, X_test, y_test)
print(model3)


Training StackingRegressor model for Team_score...
Accuracy for Team_score within 6 points: 67.21%
RMSE for Team_score: 5.6690

Training StackingRegressor model for Opponent_score...


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Accuracy for Opponent_score within 6 points: 68.85%
RMSE for Opponent_score: 5.3803

{'Team_score': StackingRegressor(estimators=[('bayesian_ridge', BayesianRidge()),
                              ('random_forest',
                               RandomForestRegressor(random_state=42))],
                  final_estimator=BayesianRidge()), 'Opponent_score': StackingRegressor(estimators=[('bayesian_ridge', BayesianRidge()),
                              ('random_forest',
                               RandomForestRegressor(random_state=42))],
                  final_estimator=BayesianRidge())}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [10]:
# Train and evaluate Bayesian Ridge regression models for multiple target variables
def baseModelWithHyperparameterTuning(X_train, y_train, X_test, y_test, threshold=6):
    models = {}
    
    for target in y_train.columns:
        print(f"Training model for {target}...\n")

        # Step 1: Broad parameter ranges for RandomizedSearchCV
        random_param_grid = {
            'alpha_1': np.logspace(-6, -2, 20),  # Wide range
            'alpha_2': np.logspace(-6, -2, 20),  # Wide range
            'lambda_1': np.logspace(-6, -2, 20), # Wide range
            'lambda_2': np.logspace(-6, -2, 20), # Wide range
            'tol': [1e-4, 1e-3, 1e-2],
            'max_iter': [100, 300, 500]
        }
        
        model = BayesianRidge()
        scorer = 'neg_mean_squared_error'
        
        random_search = RandomizedSearchCV(
            model, random_param_grid, scoring=scorer, cv=5, n_iter=50, verbose=1, random_state=42
        )
        random_search.fit(X_train, y_train[target])
        best_params_random = random_search.best_params_
        print(f"Best parameters from RandomizedSearchCV for {target}:", best_params_random)
        
        # Step 2: Define a narrower grid around the best parameters
        grid_param_grid = {
            'alpha_1': np.linspace(
                best_params_random['alpha_1'] / 10, best_params_random['alpha_1'] * 10, 10
            ),
            'alpha_2': np.linspace(
                best_params_random['alpha_2'] / 10, best_params_random['alpha_2'] * 10, 10
            ),
            'lambda_1': np.linspace(
                best_params_random['lambda_1'] / 10, best_params_random['lambda_1'] * 10, 10
            ),
            'lambda_2': np.linspace(
                best_params_random['lambda_2'] / 10, best_params_random['lambda_2'] * 10, 10
            ),
            'tol': [1e-4, 1e-3, 1e-2],
            'max_iter': [100, 300, 500]
        }
        
        grid_search = GridSearchCV(model, grid_param_grid, scoring=scorer, cv=5, verbose=1)
        grid_search.fit(X_train, y_train[target])
        best_params_grid = grid_search.best_params_
        print(f"Best parameters from GridSearchCV for {target}:", best_params_grid)
        
        # Step 3: Train the model with the best parameters
        final_model = BayesianRidge(**best_params_grid)
        final_model.fit(X_train, y_train[target])
        models[target] = final_model

        # Step 4: Make predictions
        y_pred = final_model.predict(X_test)

        # Evaluate accuracy within the threshold
        accuracy = (abs(y_pred - y_test[target]) <= threshold).mean() * 100
        print(f"Accuracy for {target} within {threshold} points: {accuracy:.2f}%")

        # Evaluate RMSE
        rmse = mean_squared_error(y_test[target], y_pred, squared=False)
        print(f"RMSE for {target}: {rmse:.4f}\n")

    return models

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the model
model_main = baseModelWithHyperparameterTuning(X_train, y_train, X_test, y_test)
print(model_main)


Training model for Team_score...

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters from RandomizedSearchCV for Team_score: {'tol': 0.01, 'max_iter': 500, 'lambda_2': 0.01, 'lambda_1': 2.9763514416313192e-05, 'alpha_2': 0.0008858667904100823, 'alpha_1': 2.9763514416313192e-05}
Fitting 5 folds for each of 90000 candidates, totalling 450000 fits
Best parameters from GridSearchCV for Team_score: {'alpha_1': 0.00029763514416313193, 'alpha_2': 8.858667904100823e-05, 'lambda_1': 2.9763514416313194e-06, 'lambda_2': 0.1, 'max_iter': 100, 'tol': 0.01}
Accuracy for Team_score within 6 points: 67.21%
RMSE for Team_score: 5.6707

Training model for Opponent_score...

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Best parameters from RandomizedSearchCV for Opponent_score: {'tol': 0.001, 'max_iter': 300, 'lambda_2': 0.01, 'lambda_1': 2.9763514416313192e-05, 'alpha_2': 1.8329807108324375e-05, 'alpha_1': 2.9763514416313192e-05}
Fitting 5 folds for each of 90000 candidates, totalling 450000 fits
Best parameters from GridSearchCV for Opponent_score: {'alpha_1': 0.00029763514416313193, 'alpha_2': 1.8329807108324375e-06, 'lambda_1': 2.9763514416313194e-06, 'lambda_2': 0.1, 'max_iter': 100, 'tol': 0.01}
Accuracy for Opponent_score within 6 points: 68.85%
RMSE for Opponent_score: 5.4268

{'Team_score': BayesianRidge(alpha_1=0.00029763514416313193, alpha_2=8.858667904100823e-05,
              lambda_1=2.9763514416313194e-06, lambda_2=0.1, max_iter=100,
              tol=0.01), 'Opponent_score': BayesianRidge(alpha_1=0.00029763514416313193, alpha_2=1.8329807108324375e-06,
              lambda_1=2.9763514416313194e-06, lambda_2=0.1, max_iter=100,
              tol=0.01)}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
